# Fine-tuning FunctionGemma pour Home Assistant

Ce notebook permet d'entraîner **FunctionGemma-270m-it** sur Google Colab avec des optimisations avancées.

**Améliorations:**
- Métriques personnalisées (précision function calls, entity accuracy)
- Scheduler cosine avec warmup
- Early stopping intelligent
- TensorBoard logging
- LoRA rank optimisé

**Pattern multi-turn:**
1. User demande une action
2. Model appelle `get_entities` pour récupérer les entités du bon domaine
3. Tool retourne les entités disponibles
4. Model appelle l'action avec la bonne entité

**Instructions:**
1. Activez le GPU: Runtime → Change runtime type → GPU
2. Exécutez les cellules dans l'ordre

## 1. Installation des dépendances

In [ ]:
# Installation des dépendances
!pip install -q transformers peft accelerate datasets bitsandbytes huggingface_hub tensorboard

In [ ]:
import torch
import gc

print(f"PyTorch version: {torch.__version__}")
print(f"GPU disponible: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"GPU: {gpu_name}")
    print(f"VRAM: {vram_gb:.1f} GB")
    
    # Recommandations automatiques basées sur le GPU
    if "A100" in gpu_name:
        print("\n✓ A100 détectée - Configuration optimale disponible")
        RECOMMENDED_BATCH = 16
        RECOMMENDED_GRAD_ACCUM = 1
    elif "V100" in gpu_name or vram_gb >= 16:
        print("\n✓ GPU 16GB+ - Bonne configuration disponible")
        RECOMMENDED_BATCH = 8
        RECOMMENDED_GRAD_ACCUM = 2
    elif "T4" in gpu_name or vram_gb >= 12:
        print("\n⚠ T4/12GB - Configuration conservative recommandée")
        RECOMMENDED_BATCH = 4
        RECOMMENDED_GRAD_ACCUM = 4
    else:
        print("\n⚠ GPU limitée - Configuration minimale")
        RECOMMENDED_BATCH = 2
        RECOMMENDED_GRAD_ACCUM = 8
    
    print(f"   Batch recommandé: {RECOMMENDED_BATCH}")
    print(f"   Gradient accumulation: {RECOMMENDED_GRAD_ACCUM}")
    print(f"   Effective batch size: {RECOMMENDED_BATCH * RECOMMENDED_GRAD_ACCUM}")

## 2. Configuration Hugging Face

FunctionGemma est un modèle gated:
1. Accepter les conditions sur https://huggingface.co/google/functiongemma-270m-it
2. Créer un token sur https://huggingface.co/settings/tokens

In [ ]:
from huggingface_hub import login
from google.colab import userdata

try:
    hf_token = userdata.get('HF_TOKEN')
    login(token=hf_token)
    print("✓ Connecté via secret Colab")
except:
    login()

## 3. Upload du dataset

In [ ]:
from google.colab import files
import os

os.makedirs("data", exist_ok=True)

print("Uploadez train.jsonl et val.jsonl")
uploaded = files.upload()

for filename in uploaded.keys():
    os.rename(filename, f"data/{filename}")
    print(f"  → data/{filename}")

In [ ]:
import json

def count_lines(filepath):
    with open(filepath, 'r') as f:
        return sum(1 for _ in f)

def analyze_dataset(filepath):
    """Analyse la distribution du dataset."""
    stats = {
        "total": 0,
        "get_entities": 0,
        "actions": {},
        "negative": 0,
    }
    
    with open(filepath, 'r') as f:
        for line in f:
            stats["total"] += 1
            example = json.loads(line)
            text = example.get('text', '')
            
            if 'get_entities' in text:
                stats["get_entities"] += 1
            
            # Détecter les actions
            for action in ['turn_on', 'turn_off', 'set_temperature', 'set_hvac_mode', 
                          'open_cover', 'close_cover', 'lock', 'unlock', 'activate']:
                if action in text:
                    stats["actions"][action] = stats["actions"].get(action, 0) + 1
            
            # Exemples négatifs
            if 'pas trouvé' in text or 'impossible' in text.lower() or 'préciser' in text:
                stats["negative"] += 1
    
    return stats

train_count = count_lines("data/train.jsonl")
val_count = count_lines("data/val.jsonl")

print(f"Dataset:")
print(f"  Train: {train_count} exemples")
print(f"  Validation: {val_count} exemples")
print(f"  Ratio val: {val_count/(train_count+val_count)*100:.1f}%")

# Analyse détaillée
print("\nAnalyse du dataset d'entraînement:")
train_stats = analyze_dataset("data/train.jsonl")
print(f"  Exemples avec get_entities: {train_stats['get_entities']} ({train_stats['get_entities']/train_stats['total']*100:.1f}%)")
print(f"  Exemples négatifs: {train_stats['negative']} ({train_stats['negative']/train_stats['total']*100:.1f}%)")
print(f"  Actions:")
for action, count in sorted(train_stats['actions'].items(), key=lambda x: -x[1]):
    print(f"    {action}: {count}")

# Aperçu
with open("data/train.jsonl", 'r') as f:
    example = json.loads(f.readline())
    print(f"\nExemple:")
    print(example['text'][:500] + "..." if len(example['text']) > 500 else example['text'])

## 4. Configuration

### Hyperparamètres optimisés

| Paramètre | Valeur | Justification |
|-----------|--------|---------------|
| LoRA rank | 64 | Meilleure capacité d'apprentissage |
| LoRA alpha | 128 | Ratio alpha/r = 2 (standard) |
| Learning rate | 1e-4 | Optimal pour LoRA fine-tuning |
| Epochs | 5 | Balance qualité/temps |
| Scheduler | Cosine | Meilleure convergence |
| Early stopping | 3 | Évite le surapprentissage |

In [ ]:
# Configuration principale
CONFIG = {
    "model_name": "google/functiongemma-270m-it",
    "max_length": 512,
    
    # LoRA - Augmenté pour meilleure capacité
    "lora_r": 64,           # Augmenté de 32 à 64
    "lora_alpha": 128,      # Ratio alpha/r = 2
    "lora_dropout": 0.05,
    "lora_target_modules": [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    
    # Entraînement - Ajuster selon GPU (voir cellule 1)
    "batch_size": RECOMMENDED_BATCH if 'RECOMMENDED_BATCH' in dir() else 8,
    "gradient_accumulation_steps": RECOMMENDED_GRAD_ACCUM if 'RECOMMENDED_GRAD_ACCUM' in dir() else 2,
    "learning_rate": 1e-4,
    "num_epochs": 5,            # Augmenté pour meilleure convergence
    "warmup_ratio": 0.1,
    "weight_decay": 0.01,
    
    # Scheduler
    "lr_scheduler_type": "cosine",  # Nouveau: scheduler cosine
    
    # Early stopping
    "early_stopping_patience": 3,   # Nouveau: arrêt après 3 eval sans amélioration
    "early_stopping_threshold": 0.01,
    
    # Sauvegarde
    "output_dir": "./output",
    "save_steps": 50,
    "logging_steps": 10,
    "eval_steps": 50,
}

# Calcul de l'effective batch size
effective_batch = CONFIG["batch_size"] * CONFIG["gradient_accumulation_steps"]

print("Configuration:")
print(f"  Model: {CONFIG['model_name']}")
print(f"  LoRA rank: {CONFIG['lora_r']} (alpha: {CONFIG['lora_alpha']})")
print(f"  Batch size: {CONFIG['batch_size']} × {CONFIG['gradient_accumulation_steps']} = {effective_batch} effective")
print(f"  Learning rate: {CONFIG['learning_rate']} ({CONFIG['lr_scheduler_type']} scheduler)")
print(f"  Epochs: {CONFIG['num_epochs']}")
print(f"  Early stopping: patience={CONFIG['early_stopping_patience']}")

## 5. Chargement du modèle

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType

print(f"Chargement de {CONFIG['model_name']}...")

model = AutoModelForCausalLM.from_pretrained(
    CONFIG["model_name"],
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(
    CONFIG["model_name"],
    trust_remote_code=True,
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = tokenizer.eos_token_id

print(f"✓ Modèle chargé! Paramètres: {model.num_parameters():,}")

In [ ]:
# Configuration LoRA optimisée
lora_config = LoraConfig(
    r=CONFIG["lora_r"],
    lora_alpha=CONFIG["lora_alpha"],
    lora_dropout=CONFIG["lora_dropout"],
    target_modules=CONFIG["lora_target_modules"],
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Activer gradient checkpointing pour économiser la mémoire
model.gradient_checkpointing_enable()
print("✓ Gradient checkpointing activé")

## 6. Préparation du dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files={
        "train": "data/train.jsonl",
        "validation": "data/val.jsonl",
    }
)

print(f"Train: {len(dataset['train'])} | Val: {len(dataset['validation'])}")

In [ ]:
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=CONFIG["max_length"],
    )
    # Pour le causal LM, les labels sont les mêmes que input_ids
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("Tokenization...")
tokenized_dataset = dataset.map(
    tokenize_function,
    remove_columns=["text"],
    batched=True,
    desc="Tokenizing",
)
print("✓ Tokenization terminée")

## 7. Métriques personnalisées

Évaluation spécifique aux function calls:
- **Function Call Accuracy**: Le modèle appelle-t-il la bonne fonction?
- **Entity Accuracy**: Le modèle sélectionne-t-il la bonne entité?
- **Negative Detection**: Le modèle détecte-t-il les requêtes impossibles?

In [ ]:
import re
import numpy as np
from transformers import EvalPrediction

def extract_function_call(text):
    """Extrait le nom de fonction et les paramètres d'un appel."""
    # Pattern: function_name(param1="value1", ...)
    match = re.search(r'(\w+)\s*\(([^)]*)\)', text)
    if match:
        func_name = match.group(1)
        params_str = match.group(2)
        
        # Extraire entity_id si présent
        entity_match = re.search(r'entity_id\s*=\s*["\']([^"\']+)["\']', params_str)
        entity_id = entity_match.group(1) if entity_match else None
        
        return func_name, entity_id
    return None, None

def compute_metrics(eval_pred: EvalPrediction):
    """Calcule les métriques personnalisées."""
    predictions, labels = eval_pred
    
    # Décoder les prédictions
    if isinstance(predictions, tuple):
        predictions = predictions[0]
    
    # Pour la perplexité, on calcule la loss moyenne
    # Note: Les métriques de function call nécessitent une génération complète
    # qui est faite séparément dans l'évaluation détaillée
    
    # Calculer la perplexité à partir des logits
    shift_logits = predictions[..., :-1, :]
    shift_labels = labels[..., 1:]
    
    # Masquer les tokens de padding (-100)
    mask = shift_labels != -100
    
    if mask.sum() > 0:
        # Calculer la cross-entropy
        from torch.nn import CrossEntropyLoss
        loss_fct = CrossEntropyLoss(reduction='none')
        
        flat_logits = torch.tensor(shift_logits).view(-1, shift_logits.shape[-1])
        flat_labels = torch.tensor(shift_labels).view(-1)
        
        losses = loss_fct(flat_logits, flat_labels)
        masked_losses = losses * mask.view(-1).float()
        
        avg_loss = masked_losses.sum() / mask.sum()
        perplexity = torch.exp(avg_loss).item()
    else:
        perplexity = float('inf')
    
    return {
        "perplexity": perplexity,
    }

print("✓ Métriques personnalisées définies")

## 8. Configuration de l'entraînement

In [ ]:
from transformers import (
    TrainingArguments, 
    Trainer, 
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback,
)

training_args = TrainingArguments(
    output_dir=CONFIG["output_dir"],
    
    # Epochs et batch
    num_train_epochs=CONFIG["num_epochs"],
    per_device_train_batch_size=CONFIG["batch_size"],
    per_device_eval_batch_size=CONFIG["batch_size"],
    gradient_accumulation_steps=CONFIG["gradient_accumulation_steps"],
    
    # Optimisation
    learning_rate=CONFIG["learning_rate"],
    lr_scheduler_type=CONFIG["lr_scheduler_type"],  # Cosine scheduler
    warmup_ratio=CONFIG["warmup_ratio"],
    weight_decay=CONFIG["weight_decay"],
    max_grad_norm=1.0,
    
    # Logging
    logging_dir="./logs",
    logging_steps=CONFIG["logging_steps"],
    report_to=["tensorboard"],  # Activer TensorBoard
    
    # Évaluation
    eval_strategy="steps",
    eval_steps=CONFIG["eval_steps"],
    
    # Sauvegarde
    save_strategy="steps",
    save_steps=CONFIG["save_steps"],
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    
    # Performance
    bf16=True,
    dataloader_num_workers=2,
    gradient_checkpointing=True,
    
    # Misc
    remove_unused_columns=False,
    seed=42,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False,
)

# Callbacks
callbacks = [
    EarlyStoppingCallback(
        early_stopping_patience=CONFIG["early_stopping_patience"],
        early_stopping_threshold=CONFIG["early_stopping_threshold"],
    )
]

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    callbacks=callbacks,
)

print(f"\n{'='*50}")
print(f"Configuration d'entraînement:")
print(f"  Epochs: {CONFIG['num_epochs']}")
print(f"  Batch: {CONFIG['batch_size']} × {CONFIG['gradient_accumulation_steps']} = {effective_batch}")
print(f"  LR: {CONFIG['learning_rate']} ({CONFIG['lr_scheduler_type']})")
print(f"  Early stopping: patience={CONFIG['early_stopping_patience']}")
print(f"  TensorBoard: ./logs")
print(f"{'='*50}")

In [ ]:
# Lancer TensorBoard (optionnel)
%load_ext tensorboard
%tensorboard --logdir ./logs

## 9. Entraînement

In [ ]:
print("Début de l'entraînement...")
print(f"  Train: {len(tokenized_dataset['train'])} exemples")
print(f"  Val: {len(tokenized_dataset['validation'])} exemples")
print()

train_result = trainer.train()

print("\n" + "="*50)
print("Entraînement terminé!")
print(f"  Training loss: {train_result.training_loss:.4f}")
print(f"  Steps: {train_result.global_step}")
print("="*50)

In [ ]:
# Évaluation finale
print("Évaluation finale...")
eval_results = trainer.evaluate()

print(f"\nRésultats:")
print(f"  Eval loss: {eval_results['eval_loss']:.4f}")
print(f"  Perplexity: {np.exp(eval_results['eval_loss']):.2f}")

## 10. Évaluation détaillée des Function Calls

Test de la qualité des prédictions sur des exemples spécifiques.

In [ ]:
import re
from collections import defaultdict

def generate_response(query: str, max_tokens: int = 150):
    """Génère une réponse du modèle."""
    text = f"<start_of_turn>user\n{query}<end_of_turn>\n<start_of_turn>model\n"
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    
    if "<start_of_turn>model" in response:
        response = response.split("<start_of_turn>model")[-1]
    if "<end_of_turn>" in response:
        response = response.split("<end_of_turn>")[0]
    
    return response.strip()

def evaluate_function_calls(test_cases):
    """Évalue la précision des function calls."""
    results = {
        "total": 0,
        "correct_function": 0,
        "correct_domain": 0,
        "has_entity": 0,
        "details": []
    }
    
    for test in test_cases:
        query = test["query"]
        expected_func = test.get("expected_function")
        expected_domain = test.get("expected_domain")
        
        response = generate_response(query)
        func_name, entity_id = extract_function_call(response)
        
        results["total"] += 1
        
        # Vérifier la fonction
        if expected_func and func_name == expected_func:
            results["correct_function"] += 1
        
        # Vérifier le domaine (pour get_entities)
        if expected_domain and func_name == "get_entities":
            if expected_domain in response:
                results["correct_domain"] += 1
        
        # Vérifier si entity_id présent
        if entity_id:
            results["has_entity"] += 1
        
        results["details"].append({
            "query": query,
            "response": response,
            "function": func_name,
            "entity": entity_id,
            "expected": expected_func,
        })
    
    return results

# Définir les cas de test
test_cases = [
    {"query": "Allume la lumière du salon", "expected_function": "get_entities", "expected_domain": "light"},
    {"query": "Éteins la lumière de la cuisine", "expected_function": "get_entities", "expected_domain": "light"},
    {"query": "Mets le chauffage à 21 degrés", "expected_function": "get_entities", "expected_domain": "climate"},
    {"query": "Ferme les volets du salon", "expected_function": "get_entities", "expected_domain": "cover"},
    {"query": "Active la scène cinéma", "expected_function": "get_entities", "expected_domain": "scene"},
    {"query": "Verrouille la porte d'entrée", "expected_function": "get_entities", "expected_domain": "lock"},
    {"query": "Allume le ventilateur", "expected_function": "get_entities", "expected_domain": "fan"},
    {"query": "Éteins tout", "expected_function": "get_entities", "expected_domain": "light"},
]

print("Évaluation des function calls...\n")
results = evaluate_function_calls(test_cases)

print(f"Résultats ({results['total']} tests):")
print(f"  Fonction correcte: {results['correct_function']}/{results['total']} ({results['correct_function']/results['total']*100:.1f}%)")
print(f"  Domaine correct: {results['correct_domain']}/{results['total']} ({results['correct_domain']/results['total']*100:.1f}%)")
print(f"  Avec entity_id: {results['has_entity']}/{results['total']}")

print("\nDétails:")
for detail in results["details"]:
    status = "✓" if detail["function"] == detail["expected"] else "✗"
    print(f"  {status} {detail['query'][:40]}...")
    print(f"      → {detail['function']}({detail['entity'] or '...'})")

## 11. Sauvegarde

In [ ]:
final_path = f"{CONFIG['output_dir']}/final"
trainer.save_model(final_path)
tokenizer.save_pretrained(final_path)

# Sauvegarder les métriques
import json
metrics = {
    "train_loss": train_result.training_loss,
    "eval_loss": eval_results['eval_loss'],
    "perplexity": float(np.exp(eval_results['eval_loss'])),
    "config": CONFIG,
    "function_call_accuracy": results['correct_function'] / results['total'] if results['total'] > 0 else 0,
}

with open(f"{final_path}/training_metrics.json", 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"✓ Modèle sauvegardé: {final_path}")
print(f"✓ Métriques sauvegardées: {final_path}/training_metrics.json")

In [ ]:
import shutil
shutil.make_archive("functiongemma-ha", 'zip', final_path)
files.download("functiongemma-ha.zip")
print("✓ Téléchargement du modèle...")

## 12. Test du modèle

**IMPORTANT:** Le format de test doit correspondre EXACTEMENT au format d'entraînement.

In [ ]:
def test_model(query: str):
    """Test le modèle - format IDENTIQUE à l'entraînement."""
    response = generate_response(query)
    return response

# Tests avec des requêtes similaires au dataset
test_queries = [
    "Allume la lumière du salon",
    "Éteins la lumière de la cuisine",
    "Mets le chauffage à 21 degrés",
    "Active le thermostat",
    "Active la scène cinéma",
    "Ferme les volets",
    # Avec typos
    "alume la lumiere du salon",
    "etein la cuisine",
]

print("Tests du modèle fine-tuné:\n")
for query in test_queries:
    print(f"User: {query}")
    response = test_model(query)
    print(f"Model: {response}")
    print()

## 13. Test multi-turn complet

Simuler le flow complet: query → get_entities → tool response → action

In [ ]:
def test_multiturn(query: str, fake_entities: str):
    """Test le flow multi-turn complet."""
    
    # Étape 1: Requête utilisateur → get_entities
    text1 = f"<start_of_turn>user\n{query}<end_of_turn>\n<start_of_turn>model\n"
    inputs1 = tokenizer(text1, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        out1 = model.generate(**inputs1, max_new_tokens=80, do_sample=False, pad_token_id=tokenizer.eos_token_id)
    
    resp1 = tokenizer.decode(out1[0], skip_special_tokens=False)
    if "<start_of_turn>model" in resp1:
        resp1 = resp1.split("<start_of_turn>model")[-1]
    if "<end_of_turn>" in resp1:
        resp1 = resp1.split("<end_of_turn>")[0]
    resp1 = resp1.strip()
    
    print(f"User: {query}")
    print(f"Model (step 1): {resp1}")
    
    # Étape 2: Injecter la réponse tool → action finale
    text2 = (
        f"<start_of_turn>user\n{query}<end_of_turn>\n"
        f"<start_of_turn>model\n{resp1}<end_of_turn>\n"
        f"<start_of_turn>tool\n{fake_entities}<end_of_turn>\n"
        f"<start_of_turn>model\n"
    )
    inputs2 = tokenizer(text2, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        out2 = model.generate(**inputs2, max_new_tokens=80, do_sample=False, pad_token_id=tokenizer.eos_token_id)
    
    resp2 = tokenizer.decode(out2[0], skip_special_tokens=False)
    if "<start_of_turn>model" in resp2:
        resp2 = resp2.split("<start_of_turn>model")[-1]
    if "<end_of_turn>" in resp2:
        resp2 = resp2.split("<end_of_turn>")[0]
    resp2 = resp2.strip()
    
    print(f"Tool: {fake_entities[:80]}...")
    print(f"Model (step 2): {resp2}")
    print()

# Tests
print("=" * 50)
print("Test Multi-Turn")
print("=" * 50 + "\n")

test_multiturn(
    "Allume la lumière du salon",
    "Entités light disponibles: light.salon, light.cuisine, light.chambre"
)

test_multiturn(
    "Mets le chauffage à 22 degrés",
    "Entités climate disponibles: climate.thermostat_salon, climate.thermostat_bureau"
)

test_multiturn(
    "Ferme les volets de la chambre",
    "Entités cover disponibles: cover.volets_salon, cover.volets_chambre, cover.volets_cuisine"
)

## 14. Résumé et prochaines étapes

### Métriques finales

In [ ]:
print("="*60)
print("RÉSUMÉ DE L'ENTRAÎNEMENT")
print("="*60)
print(f"\nModèle: {CONFIG['model_name']}")
print(f"LoRA rank: {CONFIG['lora_r']} (alpha: {CONFIG['lora_alpha']})")
print(f"\nDataset:")
print(f"  Train: {len(tokenized_dataset['train'])} exemples")
print(f"  Validation: {len(tokenized_dataset['validation'])} exemples")
print(f"\nEntraînement:")
print(f"  Epochs: {CONFIG['num_epochs']}")
print(f"  Learning rate: {CONFIG['learning_rate']} ({CONFIG['lr_scheduler_type']})")
print(f"  Batch size: {effective_batch} (effective)")
print(f"\nRésultats:")
print(f"  Training loss: {train_result.training_loss:.4f}")
print(f"  Eval loss: {eval_results['eval_loss']:.4f}")
print(f"  Perplexity: {np.exp(eval_results['eval_loss']):.2f}")
print(f"  Function call accuracy: {results['correct_function']/results['total']*100:.1f}%")
print(f"\nFichiers:")
print(f"  Modèle: {final_path}/")
print(f"  Métriques: {final_path}/training_metrics.json")
print(f"  Logs: ./logs/")
print("="*60)